# Word2Vec

Word2Vec feels like an incantation. In this notebook we are going to try out `gensim`'s word2vec implementation.

In [3]:
# Imports, Functions, Stopwords
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import sent_tokenize
import gensim
import operator

# stopwords = set(stopwords.words('english'))

# Loading the Data in a gendered partitioned fashion: 
talks_male = pd.read_csv('talks_male.csv', index_col='Talk_ID')
talks_female = pd.read_csv('talks_female.csv', index_col='Talk_ID')
talks_nog = pd.read_csv('talks_nog.csv', index_col='Talk_ID')
all_talks = pd.concat([talks_male,talks_female,talks_nog])

texts = all_talks.text.tolist()

print(f"From our {all_talks.shape[0]}x{all_talks.shape[1]} CSV, \
we have a list of {len(texts)} talks.")
talks = all_talks.text.tolist()

From our 992x14 CSV, we have a list of 992 talks.


## Building a Word Embedding Model

In his discussion of the `gensim` implementation, Radim Řehůřek notes that it expects a list of sentences as input.[[1](https://rare-technologies.com/word2vec-tutorial/)]. Because individual texts do not matter to/for `word2vec`, we are going to bundle all our sentences into a single string, then break it into sentences using the NLTK sentence tokenizer. We will preprocess the sentences to make them lowercase and to remove stop words. 

**Skip to marker.>>**

In [2]:
# def sentencer(sentence):
#     global stopwords
#     tokens = word_tokenize(sentence)
#     sentenced = [token for token in tokens if token not in stopwords and  len(token)>2]
#     return sentenced

In [3]:
# all_talks = ' '.join(talks).lower()

In [4]:
# raw = sent_tokenize(all_talks)
# # Check our work by getting the number of sentences and three sentences
# print(len(raw), raw[500:503])

220118 ['antiquated zoning and land-use regulations are still used to this day to continue putting polluting facilities in my neighborhood.', 'are these factors taken into consideration when land-use policy is decided?', 'what costs are associated with these decisions?']


In [5]:
# sentences = [sentencer(sentence) for sentence in raw]
# print(len(sentences), sentences[500:503])

220118 [['antiquated', 'zoning', 'land-use', 'regulations', 'still', 'used', 'day', 'continue', 'putting', 'polluting', 'facilities', 'neighborhood'], ['factors', 'taken', 'consideration', 'land-use', 'policy', 'decided'], ['costs', 'associated', 'decisions']]


**>>Skip to here.**

In [ ]:
# KK's 

In [4]:
w2v_model = gensim.models.Word2Vec(min_count=5, window=5, workers=3)

In [5]:
w2v_model.build_vocab(talks)

In [7]:
w2v_model.train(talks, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(68669777, 335741640)

In [9]:
w2v_model.wv.most_similar(positive=["weather"])

KeyError: "Key 'weather' not present"

In [14]:
# print( f"Model has {len(w2v_model)} terms." )

# w2v_model.save("../output/w2v_model.bin")

# To re-load this model, run
# w2v_model = gensim.models.Word2Vec.load("w2v-model.bin")